edwith의 부스트 코스 : "파이토치로 시작하는 딥러닝 기초" 를 바탕으로 작성되었습니다.  
https://www.boostcourse.org/ai214

# Loading data
- "minibatch" Gradient Descent
- PyTorch Dataset and DataLoader


매우 간단한 모델을 학습시킬때는 데이터의 양이 적어도 괜찮지만,  
복잡한 머신러닝 모델들을 학습하려면 엄청난 양의 데이터가 필요함. 현재 존재하는 dataset은 굉장히 많은 데이터들을 가지고 있는데  
비전 쪽 분야에서 가장 널리 알려진 IMAGENET 같은 dataset 은 천만장이 넘는 이미지가 데이터로 존재함.  
데이터가 많으면 많을 수록 모델이 많은 데이터를 학습하면서 더 좋은 예측을 할 수 있기 때문임.   
하지만 데이터가 너무 많다면 데이터의 각각의 gradient descent 를 하기 위해 cost 를 구해야하는데 이를 한번에 학습시키기는 불가능함.  

일부분의 데이터를 학습시키는 방식이 Minibatch Gradient Descent 임.  
전체 데이터를 작은 Minibatch 라는 작은 양으로 균일하게 나누어서 Minibatch 하나하나씩 학습하는 방법임.  
이렇게 하면 모든 데이터의 cost 를 다 계산한 후 gradient descent를 하는 것이 아니라, 각 minibatch 에 있는 데이터의 cost만 계산한 후 gradient descent를 하기 때문에  
한 번 업데이트마다 계산한 cost 의 양은 줄어들고 그에 따른 업데이트 주기가 빨라지게 됨.   

## PyTorch Dataset
PyTorch Dataset 은 PyTorch에서 제공하는 모듈로 이 모듈을 상속해 새로운 클래스를 만드는 방식으로 원하는 dataset을 지정할 수 있게 됨.  
이렇게 custom으로 dataset 클래스를 만들게 되면 2가지의 magic 메소드를 구현해야하는데  
1. dataset의 총 데이터 개수를 반환하는 `__len__()`이라는 메서드
2. 인덱스를 입력받았을때, 인덱스에 해당하는 데이터를 반환하는 `__getitem__()`이라는 메서드  
가 필요함.  `len`에서는 dataset의 총 데이터 개수를 반환하고, `getitem` 에서는 인덱스를 받아 그에 맞는 입력과 출력 데이터를 찾은 후 torch.tensor 형태로 바꾸어서 반환함. 

In [6]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = [[73, 80, 75],
                       [93, 88, 93],
                       [89, 91, 90], 
                       [96, 98, 100],
                       [73, 66, 70]]
        self.y_data = [[152], [185], [180], [196], [142]]
        
        
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        return x, y
    
dataset = CustomDataset()

## DataLoader
dataloader 의 인스턴스를 만들려면 2개를 지정해야하는데,  
1. 위에서 생성한 dataset 이고,  
2. 각 minibatch 의 크기임.  
통상적으로 minibatch의 크기는 2의 제곱수로 설정함.   
한가지 옵션을 추가해 shuffle=True 로 매번 데이터가 학습되는 순서를 섞어주는 옵션임.  
이를 설정함으로써 모델이 dataset의 순서를 외우지 못하게 방지할 수 있으므로, 주로 권장되는 옵션임.  

In [9]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset,
                       batch_size=2,
                       shuffle=True)

## Full Code

epoch 안에 minibatch 를 위한 for문 하나가 더 추가 되었는데 이때 enumerate(dataloader) 를 사용함.  
이것은 minibatch 의 인덱스, 몇 번째 minibatch 인지, 와 minibatch_size에 의해 나누어진 데이터를 줌.  

In [14]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

class Multivariate_Linear_RegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)
        
    def forward(self, x):
        return self.linear(x)
    
model = Multivariate_Linear_RegressionModel()

optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(1, nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader):
        x_train, y_train = samples
        
        # H(x) 계산
        hypothesis = model(x_train)

        # Cost 계산
        cost = F.mse_loss(hypothesis, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch : {:4d}/{} batch: {}/{} Cost : {:.6f}'.format(epoch, nb_epochs, batch_idx+1, len(dataloader), cost.item()))

Epoch :    1/20 batch: 1/3 Cost : 23414.621094
Epoch :    1/20 batch: 2/3 Cost : 3205.951416
Epoch :    1/20 batch: 3/3 Cost : 2376.525391
Epoch :    2/20 batch: 1/3 Cost : 542.302368
Epoch :    2/20 batch: 2/3 Cost : 123.560890
Epoch :    2/20 batch: 3/3 Cost : 27.622614
Epoch :    3/20 batch: 1/3 Cost : 18.889156
Epoch :    3/20 batch: 2/3 Cost : 2.911759
Epoch :    3/20 batch: 3/3 Cost : 7.161684
Epoch :    4/20 batch: 1/3 Cost : 0.999741
Epoch :    4/20 batch: 2/3 Cost : 0.891147
Epoch :    4/20 batch: 3/3 Cost : 0.625923
Epoch :    5/20 batch: 1/3 Cost : 1.152383
Epoch :    5/20 batch: 2/3 Cost : 0.857426
Epoch :    5/20 batch: 3/3 Cost : 0.055068
Epoch :    6/20 batch: 1/3 Cost : 1.027473
Epoch :    6/20 batch: 2/3 Cost : 0.338722
Epoch :    6/20 batch: 3/3 Cost : 1.558497
Epoch :    7/20 batch: 1/3 Cost : 1.033177
Epoch :    7/20 batch: 2/3 Cost : 0.374559
Epoch :    7/20 batch: 3/3 Cost : 1.304439
Epoch :    8/20 batch: 1/3 Cost : 0.103729
Epoch :    8/20 batch: 2/3 Cost : 2.02